In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import SimpleGraphVoltDatasetLoader, read_and_prepare_data
from torch_geometric_temporal.signal import temporal_signal_split
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN
from tqdm import tqdm


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


/usr/local/lib/python3.10/site-packages/torch/_tensor_str.py:137: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:283.)
  nonzero_finite_vals = torch.masked_select(


In [70]:
trafo_id = "T1330"
epochs = 25
num_timesteps_in = 12
num_timesteps_out = 4
train_ratio = 0.7
test_ratio_vs_eval_ratio = 0.5
learning_rate = 0.01
mlp_hidden_size = 4
device_str = 'cpu'

#----------------------
if device_str == 'cpu':
    torch.cuda.empty_cache()

#get dateime string of now
now = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")

In [4]:
print("Loading data...")
loader = SimpleGraphVoltDatasetLoader(trafo_id)
loader_data = loader.get_dataset(num_timesteps_in=num_timesteps_in, num_timesteps_out=num_timesteps_out)

train_dataset, test_eval_dataset = temporal_signal_split(loader_data, train_ratio=train_ratio)
test_dataset, eval_dataset = temporal_signal_split(test_eval_dataset, train_ratio=test_ratio_vs_eval_ratio)

Loading data...
Voltage index: 5
Voltage index: 5


In [71]:
class FeatureMLP(torch.nn.Module):
    def __init__(self, input_size,num_timesteps_in):
        super(FeatureMLP, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_size*num_timesteps_in, num_timesteps_in),
            #torch.nn.ReLU(),                        # we can add another layer
            #torch.nn.Linear(hidden_size, 1),        # we can add another layer
            torch.nn.Sigmoid() 
        )

    def forward(self, x):
        return self.mlp(x)

In [72]:
FeatureMLP(26,12)

FeatureMLP(
  (mlp): Sequential(
    (0): Linear(in_features=312, out_features=12, bias=True)
    (1): Sigmoid()
  )
)

In [77]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods, num_timesteps_in):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        out_channels = 32
        self.tgnn = A3TGCN(in_channels=node_features, 
                           out_channels=out_channels, 
                           periods=periods)
        
        self.feature_mlp = FeatureMLP(node_features,num_timesteps_in)

        # Equals single-shot prediction
        self.linear = torch.nn.Linear(out_channels,periods)

    def forward(self, x, edge_index, edge_weights):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        edge_weights = Graph edge weights
        """

        weighted_x = torch.zeros(x.size(0),x.size(1),x.size(2))

        #print(weighted_x.shape)
        for i in range(x.size(0)):
                
            node = x[i,:,:]
            #print(node.shape)
            reshaped_node = node.reshape(1,-1)
            #print(reshaped_node.shape)

            feature_weights = self.feature_mlp(reshaped_node)
            #print(feature_weights.shape)

            weighted_x[i] = node * feature_weights

        #we can do this for edge_weights too...

        h = self.tgnn(weighted_x, edge_index, edge_weights)
        h = F.relu(h)
        h = self.linear(h)
        return h


In [78]:
def train_test(model, feature_mlp,device, train_dataset, test_dataset, optimizer, loss_fn, epochs, now):
    """
    Definition of the training loop.
    """
    epoch_losses_train = []
    epoch_losses_test = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss_train = 0

        subset = 100
        step=0

        for snapshot in tqdm(train_dataset, desc="Training epoch {}".format(epoch)):
            snapshot.to(device)

            #create a tensor of the same size as the snapshot.x where I will store the weighted x
            weighted_x = torch.zeros(snapshot.x.size(0),snapshot.x.size(1),snapshot.x.size(2))

            for i in range(snapshot.x.size(0)):
                
                node = snapshot.x[i,:,:]
                #print(node.shape)
                reshaped_node = node.reshape(1,-1)
                #print(reshaped_node.shape)

                feature_weights = feature_mlp(reshaped_node)
                #print(feature_weights.shape)

                weighted_x[i] = node * feature_weights

            #print(weighted_x.shape)

            optimizer.zero_grad()
            out = model(weighted_x, snapshot.edge_index,snapshot.edge_weight)
            loss = loss_fn()(out, snapshot.y)
            loss.backward()
            optimizer.step()
            epoch_loss_train += loss.detach().cpu().numpy()

            step+=1

            if step > subset:
                break


        epoch_losses_train.append(epoch_loss_train)
        model.eval()
        epoch_loss_test = 0
        with torch.no_grad():

            subset = 100
            step=0

            for snapshot in tqdm(test_dataset, desc="Testing epoch {}".format(epoch)):
                snapshot.to(device)

                weighted_x = torch.zeros(snapshot.x.size(0),snapshot.x.size(1),snapshot.x.size(2))
                for i in range(snapshot.x.size(0)): 
                    node = snapshot.x[i,:,:]
                    reshaped_node = node.reshape(1,-1)
                    feature_weights = feature_mlp(reshaped_node)

                    weighted_x[i] = node * feature_weights

                out = model(weighted_x, snapshot.edge_index,snapshot.edge_weight)
                loss = loss_fn()(out, snapshot.y).cpu().numpy()
                epoch_loss_test += loss

                step+=1
                if step > subset:
                    break

            epoch_losses_test.append(epoch_loss_test)
            if min(epoch_losses_test) == epoch_loss_test:
                torch.save(model.state_dict(), f"../models/A3TGCN_{now}_{trafo_id}_epochs-{epochs}_in-{num_timesteps_in}_out-{num_timesteps_out}_train-ratio-{train_ratio}_lr-{learning_rate}.pt")
            print("Epoch: {}, Train Loss: {:.7f}, Test Loss: {:.7f}".format(epoch, epoch_loss_train, epoch_loss_test))
        
        
    return epoch_losses_train, epoch_losses_test
            

In [79]:
def eval(model, feature_mlp,eval_dataset, device, loss_fn, std):
    with torch.no_grad():
        model.eval()
        loss_all = 0
        loss_elementwise = 0
        
        steps = 0
        for snapshot in tqdm(eval_dataset, desc="Evaluating"):
            steps += 1
            snapshot.to(device)

            weighted_x = torch.zeros(snapshot.x.size(0),snapshot.x.size(1),snapshot.x.size(2))
            for i in range(snapshot.x.size(0)): 
                node = snapshot.x[i,:,:]
                reshaped_node = node.reshape(1,-1)
                feature_weights = feature_mlp(reshaped_node)

                weighted_x[i] = node * feature_weights

            out = model(weighted_x, snapshot.edge_index,snapshot.edge_weight)
            loss_all += loss_fn()(out, snapshot.y).cpu().numpy()
            loss_elementwise += loss_fn(reduction="none")(out, snapshot.y).cpu().numpy()

            if steps > 1000:
                break

        loss_all *= std/steps
        loss_elementwise *= std/steps
    return loss_all, loss_elementwise

In [80]:
print("Running training...")
device = torch.device(device_str)
feature_mlp = FeatureMLP(train_dataset[0].x.shape[1],num_timesteps_in=num_timesteps_in).to(device)
model = TemporalGNN(node_features=train_dataset[0].x.shape[1], periods=train_dataset[0].y.shape[1],num_timesteps_in=num_timesteps_in).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.L1Loss
losses = train_test(model, feature_mlp, device, train_dataset, test_dataset, optimizer, loss_fn, epochs=epochs, now=now)

Running training...


Training epoch 0: 100it [00:05, 17.18it/s]
Testing epoch 0: 100it [00:01, 52.22it/s]


Epoch: 0, Train Loss: 27.6634571, Test Loss: 53.1920792


Training epoch 1: 100it [00:05, 18.15it/s]
Testing epoch 1: 100it [00:01, 57.75it/s]


Epoch: 1, Train Loss: 30.1545750, Test Loss: 53.2614544


...

Epoch: 19, Train Loss: 23.3630729, Test Loss: 52.8386805


Training epoch 20: 100it [00:06, 15.12it/s]
Testing epoch 20: 100it [00:02, 44.63it/s]


Epoch: 20, Train Loss: 22.5057907, Test Loss: 53.3432799


Training epoch 21: 44it [00:03, 13.67it/s]


KeyboardInterrupt: 

In [65]:
print(losses)

std = loader.mean_and_std["measurements"][1]["voltage"]

#read saved model
model.load_state_dict(torch.load(f"../models/A3TGCN_{now}_{trafo_id}_epochs-{epochs}_in-{num_timesteps_in}_out-{num_timesteps_out}_train-ratio-{train_ratio}_lr-{learning_rate}.pt"))

loss_all, loss_elementwise = eval(model, eval_dataset, device, loss_fn, std)

print("Loss all: {:.7f}".format(loss_all))
print("Loss elementwise: {}".format(loss_elementwise))

NameError: name 'losses' is not defined